In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'


from torchvision import models
model_names = [name for name in dir(models) if callable(getattr(models, name))]
print(model_names)
# 사전 학습된 모델 로드

OUTFILE1 = './seed4/ft_efficientnet_v2_m_submit_seed_'
OUTFILE2 = '.csv'


['AlexNet', 'DenseNet', 'EfficientNet', 'GoogLeNet', 'GoogLeNetOutputs', 'Inception3', 'InceptionOutputs', 'MNASNet', 'MobileNetV2', 'MobileNetV3', 'RegNet', 'ResNet', 'ShuffleNetV2', 'SqueezeNet', 'VGG', '_GoogLeNetOutputs', '_InceptionOutputs', 'alexnet', 'densenet121', 'densenet161', 'densenet169', 'densenet201', 'efficientnet_b0', 'efficientnet_b1', 'efficientnet_b2', 'efficientnet_b3', 'efficientnet_b4', 'efficientnet_b5', 'efficientnet_b6', 'efficientnet_b7', 'googlenet', 'inception_v3', 'mnasnet0_5', 'mnasnet0_75', 'mnasnet1_0', 'mnasnet1_3', 'mobilenet_v2', 'mobilenet_v3_large', 'mobilenet_v3_small', 'regnet_x_16gf', 'regnet_x_1_6gf', 'regnet_x_32gf', 'regnet_x_3_2gf', 'regnet_x_400mf', 'regnet_x_800mf', 'regnet_x_8gf', 'regnet_y_16gf', 'regnet_y_1_6gf', 'regnet_y_32gf', 'regnet_y_3_2gf', 'regnet_y_400mf', 'regnet_y_800mf', 'regnet_y_8gf', 'resnet101', 'resnet152', 'resnet18', 'resnet34', 'resnet50', 'resnext101_32x8d', 'resnext50_32x4d', 'shufflenet_v2_x0_5', 'shufflenet_v2_x1

In [5]:
import pandas as pd
import numpy as np
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from sklearn.ensemble import IsolationForest
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import random

# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    
    
    
for seed in range(73,80):
    seed_everything(seed) # Seed 고정
    model = models.efficientnet_b7(pretrained=True)
    # 데이터 로딩 클래스 정의
    class TrainDataset(Dataset):
        def __init__(self, csv_file, transform=None):
            """
            Args:
                csv_file (string): csv 파일의 경로.
                transform (callable, optional): 샘플에 적용될 Optional transform.
            """
            self.df = pd.read_csv(csv_file)
            self.transform = transform

        def __len__(self):
            return len(self.df)

        def __getitem__(self, idx):
            img_path = self.df['img_path'].iloc[idx]
            image = Image.open(img_path)
            label = self.df.iloc[idx, 2]  # 라벨
            if self.transform:
                image = self.transform(image)
            return image,label


    class CustomDataset(Dataset):
        def __init__(self, csv_file, transform=None):
            """
            Args:
                csv_file (string): csv 파일의 경로.
                transform (callable, optional): 샘플에 적용될 Optional transform.
            """
            self.df = pd.read_csv(csv_file)
            self.transform = transform

        def __len__(self):
            return len(self.df)

        def __getitem__(self, idx):
            img_path = self.df['img_path'].iloc[idx]
            image = Image.open(img_path)

            if self.transform:
                image = self.transform(image)
            return image

    # 이미지 전처리 및 임베딩
    transform = transforms.Compose([
        #transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])


    train_data = TrainDataset(csv_file='./bigdata/train.csv', transform=transform)
    train_loader = DataLoader(train_data, batch_size=4, shuffle=False)

    # 출력 레이어 수정
    num_ftrs = model.classifier[1].in_features

    # 이진 분류를 위한 출력 크기로 변경
    model.classifier = nn.Sequential(
        nn.Dropout(0.2),
        nn.Linear(num_ftrs, 2)
    )

    model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=5e-5)

    model.train()  # 학습 모드 설정
    for epoch in tqdm(range(500)):
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()


    torch.save(model.state_dict(), './bigdata/rsn_model/efnet_tt_model.pth')     
    model.load_state_dict(torch.load('./bigdata/rsn_model/efnet_tt_model.pth'))
    model.classifier  = nn.Identity()  # 마지막 레이어를 Identity로 설정하여 임베딩 출력



    model.eval()  # 추론 모드로 설정

    # 특성 추출을 위한 모델의 마지막 레이어 수정
    #model = torch.nn.Sequential(*(list(model.children())[:-1]))

    model.to(device)

    # 이미지를 임베딩 벡터로 변환
    def get_embeddings(dataloader, model):
        embeddings = []
        with torch.no_grad():
            for images in tqdm(dataloader):
                images = images.to(device)
                emb = model(images)
                embeddings.append(emb.cpu().numpy().squeeze())
        return np.concatenate(embeddings, axis=0)
    train_data = CustomDataset(csv_file='./bigdata/train.csv', transform=transform)
    train_loader = DataLoader(train_data, batch_size=32, shuffle=False)
    train_embeddings = get_embeddings(train_loader, model)


    # Isolation Forest 모델 학습
    clf = IsolationForest(random_state=42)
    clf.fit(train_embeddings)


    # 테스트 데이터에 대해 이상 탐지 수행
    test_data = CustomDataset(csv_file='./bigdata/test.csv', transform=transform)
    test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

    test_embeddings = get_embeddings(test_loader, model)
    test_pred = clf.predict(test_embeddings)

    # Isolation Forest의 예측 결과(이상 = -1, 정상 = 1)를 이상 = 1, 정상 = 0으로 변환
    test_pred = np.where(test_pred == -1, 1, 0)


    submit = pd.read_csv('./bigdata/sample_submission.csv')
    submit['label'] = test_pred
    for i in test_pred:
        print(i,end=",")


    submit.to_csv(OUTFILE1+str(seed)+OUTFILE2, index=False)

100%|█████████████████████████████████████████████| 4/4 [00:02<00:00,  1.67it/s]


1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,1,1,1,0,0,0,0,1,0,1,1,1,1,1,1,1,0,0,0,0,1,

100%|█████████████████████████████████████████████| 4/4 [00:02<00:00,  1.77it/s]


1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,0,0,1,0,1,1,1,1,1,0,0,0,0,1,

100%|█████████████████████████████████████████████| 4/4 [00:02<00:00,  1.83it/s]


...

1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,1,1,1,1,1,0,0,0,0,1,

100%|█████████████████████████████████████████████| 4/4 [00:02<00:00,  1.83it/s]


0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,1,1,1,1,0,0,0,0,1,

100%|█████████████████████████████████████████████| 4/4 [00:02<00:00,  1.88it/s]

1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,1,1,1,1,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,1,1,1,0,0,0,1,1,0,1,1,1,1,1,1,1,0,0,0,0,1,